In [1]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas
# !pip install ipywidgets


from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import datadetox as detox
import importlib

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [70]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 7):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

#fitragem série B comjunto 5
df_list[4] = df_list[4][df_list[4]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {  'data': 'Data','Date': 'Data','data_partida': 'Data',
                        'time_mandante': 'Mandante','mandante': 'Mandante','Home': 'Mandante',
                        'time_visitante': 'Visitante','visitante': 'Visitante','Away': 'Visitante'}
    

df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

df_list = detox.padronizar_datas(df_list)

DataFrame 0: Já é datetime64, mantido como está.
DataFrame 1: Convertendo padrão com '/'
DataFrame 2: Convertendo padrão com '/'
DataFrame 3: Convertendo padrão com '/'
DataFrame 4: Convertendo padrão ISO com '-'
DataFrame 5: Já é datetime64, mantido como está.


Unir todos os DFs com agrupamento por colunas, Data, Mandante e Visitante

In [67]:
#mapear cluber qeu o filtro BERT nao resolve (manualmente)
clubes_map_inicial = {  "America-RN":"América Natal","América-RN":"América Natal","América rn":"América Natal",
                        "Joinvile":"Joinville",
                        "Atlético pr":"Athlético-PR","Atlético-PR":"Athlético-PR", 
                        "Grêmio Prudente":"Barueri","Grêmio prudente":"Barueri","Gremio Prudente":"Barueri"}

for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_map_inicial)
    df['Visitante'] = df['Visitante'].replace(clubes_map_inicial)

# listar clubes Mandantes e visitantes, para agrupar ortograficamente
clubes_unicos = []
for df in df_list:
    unicos_por_bd = pd.unique(np.concatenate([df['Mandante'].values, df['Visitante'].values])).tolist()
    clubes_unicos = list(set(clubes_unicos) | set(unicos_por_bd))

clubes_value_dict = [   "América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                        "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                        "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                        "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                        "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                        "Sport Recife", "Vasco da Gama", "Vitória"]

modelo = SentenceTransformer('all-MiniLM-L6-v2')
clubes_corrigidos = detox.corrigir_times_semanticamente(clubes_unicos, clubes_value_dict,modelo)
clubes_dict = detox.gerar_dicionario_mapeamento(clubes_corrigidos, limite=0.0)

# corrigindo e agrupando clubes em todos os dataframes
for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_dict)
    df['Visitante'] = df['Visitante'].replace(clubes_dict)

# unindo todos os dfs com merge on=['Data','Mandante','Visitante']
df_merged = pd.DataFrame(columns=['Data','Mandante','Visitante'])
for i,df in enumerate(df_list):
    df_merged = pd.merge(df_merged, df, how='outer', on=['Data','Mandante','Visitante'],suffixes=('', f'_{i}'))


In [68]:
pd.set_option('display.max_rows', None)
import datetime as dt
df = df_merged.copy()

# Criar edições dos campeonatos visto que 2020 foi ate 2021
mask = (df['Data'] > dt.datetime(2020, 8, 1)) & (df['Data'] < dt.datetime(2021, 2, 28))
df.loc[mask, 'edition'] = 2020
df.loc[~mask, 'edition'] = df.loc[~mask, 'Data'].dt.year

df.sort_values(by=['Mandante', 'Visitante','edition','Data'], ascending=[True, True, True, True], inplace=True)
#df.head(10)

In [69]:
importlib.reload(detox)

# Agrupar e aplicar a função
df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)

# Aplica o agrupamento e mescla as linhas sem conflito
df_mesclado = (
    df_ajustado.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado = df_mesclado[df_mesclado['edition']<2024]

df_group = df_mesclado.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(
    lambda group: detox.ajustar_datas_semdiff(group,keep=True))

df_mesclado2 = (
    df_group.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado2.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx',index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_10264\2427877954.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)
C:\Users\USER\AppData\Local\Temp\ipykernel_10264\2427877954.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(detox.mesclar_linhas_sem_conflito)
C:\Users\USER\AppData\Local\Temp\ipykernel_10264\2427877954.py:9: DeprecationWarn

Garantindo partidas por temporada

In [272]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
importlib.reload(detox)

df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx')

#ajustando sao paulo e sao caetano jogo interrompido
df.drop(1007, inplace=True)

# ajustando jogo do flamengo
df.loc[2835, 'Data'] = pd.to_datetime('2009-11-07')
df.drop(2834, inplace=True)

# ajustando jogo do BOTAFOGO
df.loc[1969, 'Data'] = pd.to_datetime('2009-07-29')
df.drop(1968, inplace=True)

# ajustando jogo da chapecoense
df.drop(5753, inplace=True)

#2005
df.drop([1137,1137,1207,1231,1257,1284,1307,1310,1328,1341,1375,1379], inplace=True)
df.loc[1138, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1208, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1232, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1258, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1285, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1308, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1311, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1329, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1342, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1376, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1380, 'Data'] = pd.to_datetime('2009-10-19')

#2007 
df.drop([1970,1971], inplace=True)


detox.verifica_partidas_edi(df,'edition','Mandante')
detox.verifica_partidas_edi(df,'edition','Visitante')

df.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx',index=False)

Edições com número desigual de partidas por time: edition
2016    2
Name: n_partidas, dtype: int64
Edições com número desigual de partidas por time: edition
2016    2
Name: n_partidas, dtype: int64


Meslclando Colunas

In [2]:
df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx')

gols_mandante = ["gols_mandante","mandante_Placar","Mandante Placar","mandante_Placar_3","Gol Mandante","placar_mandante"]
gols_visitante = ["gols_visitante","visitante_Placar","Visitante Placar","visitante_Placar_3","Gol Visitante","placar_visitante"]
colunas_hora = ["Horário", "hora", "hora_3", "Hora", "hora_partida"]

for col in colunas_hora:
    df[col] = df[col].str.replace(r'h', ':', regex=True)
    #display(df[col].str.match(r'^\d{2}:\d{2}$').value_counts())
    df[col] = pd.to_datetime(df[col], format='%H:%M')

df[gols_mandante].nunique(axis=1,dropna=True).value_counts()
df[gols_visitante].nunique(axis=1,dropna=True).value_counts()
df[colunas_hora].nunique(axis=1,dropna=True).value_counts()

df['gols_mandante_moda'] = df[gols_mandante].mode(axis=1).iloc[:, 0]
df['gols_visitante_moda'] = df[gols_visitante].mode(axis=1).iloc[:, 0]
df['horario'] = df[colunas_hora].mode(axis=1).iloc[:, 0]
df.drop(columns=gols_mandante+gols_visitante+colunas_hora, inplace=True)
importlib.reload(detox)

df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=1,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=2,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=3,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=4,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=5,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=6,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=7,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=8,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=9,div=30))
df.loc[2191, 'horario'] = pd.to_datetime('2009-10-19 19:00') #horario meio noite
df['horario'] = df['horario'].dt.time


arenas_col = ['arena', 'Arena', 'arena_3', 'Estádio', 'estadio']

for col in arenas_col:
    df[col] = df[col].apply(detox.normalizar_texto)
    df[col] = df[col].str.replace(r'\bestadio\b', '', regex=True).str.strip()
    df[col] = df[col].str.replace(r'\(\*pf\)', '', regex=True).str.strip()
    df[col] = df[col].str.replace(r'\*\(pf\)', '', regex=True)

mapeamento_estadios = detox.ler_json_as_dict('arenas_dict.json')

for arena in arenas_col:
    df[arena] = df[arena].replace(mapeamento_estadios)

# indicando a moda
df['Arena_moda'] = df[arenas_col].mode(axis=1, dropna=True).iloc[:, 0]

df.drop(columns=arenas_col, inplace=True)

drop_list = ["mandante_Estado", "mandante_Estado_3", "Estado Mandante", "UF", "UF Mandante", "Cidade", "Turno", "Dia da Semana", "Série"]
df.drop(columns=drop_list, inplace=True)

Feature Engineering

In [63]:
importlib.reload(detox)
#criar colunas de pontuação na rodada
df.rename(columns={'gols_mandante_moda': 'Placar_mandante', 'gols_visitante_moda': 'Placar_visitante'}, inplace=True)
df = detox.distribuir_pontos(df,'Placar_mandante','Placar_visitante')
df.sort_values(by=['Temporada','Data'],ascending=[True, True], inplace=True,ignore_index=True)
df['rodada'] = None
mask_rodada = (df['rodada'] != None)


# atribuindo rodadas ás temporadas
season_list = [2003]#,2005,2006]#list(df['Temporada'].unique())
for ano in season_list:
    mask_season = (df['Temporada'] == ano)
    num_clubes = len(df[mask_season]['Mandante'].unique())
    final_rodada = ((num_clubes*2)-1)

    for rodada in range(1,final_rodada):
        clubes_ja_jogaram = set()

        for idx, row in df[mask_season & mask_rodada].iterrows():
            mandante = row['Mandante']
            visitante = row['Visitante']
            rodada_none = row['rodada']


            if mandante not in clubes_ja_jogaram and visitante not in clubes_ja_jogaram and rodada_none == None:

                #linha = idx+((num_clubes/2)*(rodada-1))
                #print(f'L:{int(linha)} I:{idx}')
                df.loc[idx, 'rodada'] = rodada  
                clubes_ja_jogaram.add(mandante)
                clubes_ja_jogaram.add(visitante)

            if len(clubes_ja_jogaram) == num_clubes:
                print(f'rodada {rodada} completa {idx}')
                break
            
            df = df[mask_rodada]

#print(df['rodada'].value_counts())
df.to_excel("teste.xlsx",index=False)


rodada 1 completa 11
rodada 2 completa 23
rodada 3 completa 284
rodada 4 completa 275
rodada 5 completa 192
rodada 6 completa 102
rodada 7 completa 241
rodada 8 completa 345
rodada 9 completa 363
rodada 10 completa 173
rodada 11 completa 120
rodada 12 completa 399
rodada 13 completa 411
rodada 14 completa 430
rodada 15 completa 435
rodada 16 completa 453
rodada 17 completa 189
rodada 18 completa 477
rodada 19 completa 213
rodada 20 completa 263
rodada 21 completa 513
rodada 22 completa 524
rodada 23 completa 260
rodada 24 completa 528
rodada 26 completa 299
rodada 27 completa 311
rodada 28 completa 323
rodada 29 completa 335
rodada 31 completa 359
rodada 33 completa 383
rodada 34 completa 395
rodada 40 completa 467
rodada 42 completa 491
rodada 43 completa 503


In [65]:
pd.set_option('display.max_rows', None)
display(df[mask_season][['Temporada','Data','Mandante','Visitante','Pontos Mandante','Pontos Visitante','rodada']])
print(df[df['Temporada'] == 2003]['rodada'].value_counts())

,Temporada,Data,Mandante,Visitante,Pontos Mandante,Pontos Visitante,rodada
0,2003,2003-03-29,Athletico Paranaense,Grêmio,3,0,1
1,2003,2003-03-29,Guarani,Vasco da Gama,3,0,1
2,2003,2003-03-30,Corinthians,Atlético Mineiro,0,3,1
3,2003,2003-03-30,Criciúma,Fluminense,3,0,1
4,2003,2003-03-30,Cruzeiro,São Caetano,1,1,1
5,2003,2003-03-30,Flamengo,Coritiba,1,1,1
6,2003,2003-03-30,Fortaleza,Bahia,1,1,1
7,2003,2003-03-30,Goiás,Paysandu,1,1,1
8,2003,2003-03-30,Internacional,Ponte Preta,1,1,1
9,2003,2003-03-30,Juventude,São Paulo,1,1,1


rodada
1     12
2     12
3     12
4     12
5     12
6     12
7     12
8     12
9     12
10    12
11    12
12    12
13    12
14    12
15    12
16    12
17    12
18    12
19    12
20    12
21    12
22    12
23    12
24    12
26    12
27    12
29    12
28    12
33    12
31    12
42    12
43    12
34    12
40    12
30    11
25    11
35    11
32    11
38    11
37    11
36    11
39    11
41    11
44    11
45    11
46    11
Name: count, dtype: int64


In [ ]:
filtro = df[mask_season].groupby('rodada').filter(
    lambda x: x[['Mandante', 'Visitante']].drop_duplicates().shape[0] > 1
)

pd.set_option()
display(filtro)

,Temporada,Data,Mandante,Visitante,arbitro,publico,publico_max,Placar_mandante,Placar_visitante,horario,Arena_moda,Pontos Mandante,Pontos Visitante,rodada
552,2004,2004-04-21,Botafogo,Goiás,NaN,NaN,NaN,1,4,16:00:00,caio martins,0,3,1
553,2004,2004-04-21,Cruzeiro,Juventude,NaN,NaN,NaN,2,1,18:00:00,ipatingao,3,0,1
554,2004,2004-04-21,Figueirense,Internacional,NaN,NaN,NaN,1,0,16:00:00,orlando scarpelli,3,0,1
555,2004,2004-04-21,Grêmio,Flamengo,NaN,NaN,NaN,0,0,21:45:00,olimpico,1,1,1
556,2004,2004-04-21,Palmeiras,Atlético Mineiro,NaN,NaN,NaN,0,0,18:00:00,allianz parque,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,2004,2004-12-19,Guarani,Grêmio,NaN,NaN,NaN,2,0,16:00:00,brinco de ouro,3,0,45
1100,2004,2004-12-19,Internacional,Paraná,NaN,NaN,NaN,2,1,16:00:00,beira-rio,3,0,45
1101,2004,2004-12-19,Juventude,Paysandu,NaN,NaN,NaN,1,2,16:00:00,alfredo jaconi,0,3,45
1102,2004,2004-12-19,Santos,Vasco da Gama,NaN,NaN,NaN,2,1,16:00:00,teixeirao,3,0,45


In [9]:
#display(df[(df['Temporada'] == 2004) & ((df['Mandante'] == 'São Caetano') | (df['Visitante'] == 'São Caetano'))])
display(df[(df['Temporada'] == 2004) & (df['rodada'] == 2)][['Data','Mandante','Visitante','rodada']])

,Data,Mandante,Visitante,rodada
576,2004-04-28,Botafogo,Atlético Mineiro,2
577,2004-04-28,Coritiba,Internacional,2
578,2004-04-28,Cruzeiro,Criciúma,2
579,2004-04-28,Figueirense,Santos,2
580,2004-04-28,Grêmio,Corinthians,2
581,2004-04-28,Paysandu,Athletico Paranaense,2
582,2004-04-28,Ponte Preta,Goiás,2
583,2004-04-28,São Paulo,Fluminense,2
584,2004-04-29,Palmeiras,Vitória,2
585,2004-04-29,Paraná,Flamengo,2


In [89]:
for ano in range(2003,2022):
    print(ano)
    mask_season = (df['Temporada'] == ano)
    mask_rodada = df[mask_season]['rodada'].value_counts() != 12
    rodadas_incompletas = mask_rodada[mask_rodada].index.tolist()
    print(len(rodadas_incompletas))



2003
0
2004
0
2005
42
2006
38
2007
38
2008
38
2009
38
2010
38
2011
38
2012
38
2013
38
2014
38
2015
38
2016
38
2017
38
2018
38
2019
38
2020
38
2021
38


In [60]:
# atribuindo classificação por rodada e temporada
df.sort_values(by=['Temporada','rodada'],ascending=[True, True], inplace=True)
mask_season = (df['Temporada'] == 2021)
keys_club = list(df[mask_season]['Mandante'].unique())
score = dict.fromkeys(keys_club,0)

#score['Atlético Mineiro'] += 3

# Agrupa por rodada
grupo_rodada = df[mask_season].groupby('rodada')

# Itera pelos grupos
for rodada, grupo in grupo_rodada:
    for _, row in grupo.iterrows():

        # atualiza pontuação na lista para mandantes
        nome_m = row['Mandante']
        pontos_m = row['Pontos Mandante']
        score[nome_m] += pontos_m

        # atualiza pontuação na lista para visitantes
        nome_v = row['Visitante']
        pontos_v = row['Pontos Visitante']
        score[nome_v] += pontos_v

print(score)

{'Bahia': 43, 'Cuiabá': 47, 'São Paulo': 48, 'Athletico Paranaense': 47, 'Atlético Mineiro': 84, 'Ceará': 50, 'Chapecoense': 15, 'Corinthians': 57, 'Flamengo': 71, 'Internacional': 48, 'Atlético Goianiense': 53, 'Red Bull Bragantino': 56, 'Santos': 50, 'América Mineiro': 53, 'Fluminense': 54, 'Fortaleza': 58, 'Juventude': 46, 'Palmeiras': 66, 'Sport Recife': 38, 'Grêmio': 43}


In [57]:
# df_aux = df[mask_season & ((df['Mandante'] == 'São Caetano') | (df['Visitante'] == 'São Caetano'))]
# #print(df_aux['pontos'])
# #display(sorted(df_aux))
# df_aux[['Mandante','Visitante','rodada','Placar_mandante','Placar_visitante']].sort_values(by='rodada')

In [61]:
ordenado = dict(sorted(score.items(), key=lambda item: item[1]))
display(ordenado)

{'Chapecoense': 15,
 'Sport Recife': 38,
 'Bahia': 43,
 'Grêmio': 43,
 'Juventude': 46,
 'Cuiabá': 47,
 'Athletico Paranaense': 47,
 'São Paulo': 48,
 'Internacional': 48,
 'Ceará': 50,
 'Santos': 50,
 'Atlético Goianiense': 53,
 'América Mineiro': 53,
 'Fluminense': 54,
 'Red Bull Bragantino': 56,
 'Corinthians': 57,
 'Fortaleza': 58,
 'Palmeiras': 66,
 'Flamengo': 71,
 'Atlético Mineiro': 84}

In [5]:
pd.set_option('display.max_rows', 150)
#pd.set_option('display.max_columns', None)

display(df)
#display(df[['Temporada','Data','Mandante','Visitante','rodada']])

,Temporada,Data,Mandante,Visitante,arbitro,publico,publico_max,Placar_mandante,Placar_visitante,horario,Arena_moda,Pontos Mandante,Pontos Visitante,rodada
0,2003,2003-03-29,Athletico Paranaense,Grêmio,NaN,NaN,NaN,2,0,16:00:00,ligga arena,3,0,1
1,2003,2003-03-29,Guarani,Vasco da Gama,NaN,NaN,NaN,4,2,16:00:00,brinco de ouro,3,0,1
2,2003,2003-03-30,Corinthians,Atlético Mineiro,NaN,NaN,NaN,0,3,18:00:00,pacaembu,0,3,1
3,2003,2003-03-30,Criciúma,Fluminense,NaN,NaN,NaN,2,0,16:00:00,heriberto hulse,3,0,1
4,2003,2003-03-30,Cruzeiro,São Caetano,NaN,NaN,NaN,2,2,16:00:00,mineirao,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8401,2023,2023-12-06,Goiás,América Mineiro,Davi de Oliveira Lacerda,1966,14525,1,0,19:00:00,serrinha,3,0,38
8402,2023,2023-12-06,Internacional,Botafogo,Flavio Rodrigues de Souza,27365,50848,3,1,21:30:00,beira-rio,3,0,38
8403,2023,2023-12-06,Santos,Fortaleza,Leandro Pedro Vuaden,14130,20120,1,2,21:30:00,vila belmiro,0,3,38
8404,2023,2023-12-06,São Paulo,Flamengo,Caio Max Augusto Vieira,36618,72039,1,0,21:30:00,morumbi,3,0,38
